In [1]:
%matplotlib inline

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
import cv2
import numpy as np
import os

def downsample_and_classify_land_ocean(image_path, N, M):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR|cv2.IMREAD_IGNORE_ORIENTATION)
    # Plot the original image
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.show()
    if image is None:
        raise ValueError("Image not found or path is incorrect.")


    # Convert to HSV color space for easier color thresholding
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)


    # Define color range for ocean (blueish) in HSV
    ocean_lower = np.array([95, 10, 0])  # Adjust these values as needed
    ocean_upper = np.array([220, 255, 255])

    # Create a binary mask where ocean-like colors are marked as white
    ocean_mask = cv2.inRange(hsv_image, ocean_lower, ocean_upper)
    # Resize to N x M
    downsampled_map = cv2.resize(ocean_mask, (M, N), interpolation=cv2.INTER_NEAREST)

    # Convert to boolean array: True for land, False for ocean
    binary_map = downsampled_map < 128  # threshold to boolean



    return binary_map

def generate_edge_land_mask(rows, cols):
    # Find if there is image to build land_image
    image_path = next((os.path.join('./', f) for f in os.listdir('./')
                   if f.startswith("land_image") and f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))), None)
    if image_path is not None:
        # Convert a colorful image into land and ocean
        land_mask = downsample_and_classify_land_ocean(image_path, rows, cols)
        return land_mask
    # If no land_image, generate land around the edges
    land_mask = np.zeros((rows, cols), dtype=bool)

    # Set the edges as land
    land_mask[0, :] = True
    land_mask[-1, :] = True
    land_mask[:, 0] = True
    land_mask[:, -1] = True
    return land_mask

def generate_random_inputs(rows, cols):
    # Generate a random ocean map with initial concentration values
    ocean_map = np.zeros((rows, cols))

    # Generate land mask with land on the edges only
    land_mask = generate_edge_land_mask(rows, cols)

    # Generate S array for pollutant sources/treatment (random values as a simulation)
    # Assign negative values to simulate waste processing (cleaning up) at some random locations
    S = np.zeros((rows, cols))

    # Assign positive values to simulate pollutant sources at some random locations
    for _ in range(10):  # Adding 10 points for pollutant sources
        i, j = np.random.randint(1, rows-1), np.random.randint(1, cols-1)
        S[i, j] = 100  # Positive value for pollution

    return ocean_map, land_mask, S

def generate_random_Dx_Dy(rows, cols):
    Dx = np.random.rand(rows, cols) * 0
    Dy = np.random.rand(rows, cols) * 0
    u = np.random.randn(rows, cols) * 10
    v = np.random.randn(rows, cols) * 10
    return Dx, Dy, u, v

def update_concentration_parallel(ocean_map, land_mask, S, Dx, Dy, u, v, dt, dx, dy, iterations):
    rows, cols = ocean_map.shape
    maps_over_time = [np.copy(ocean_map)]
    for iteration in range(iterations):
        C_current = ocean_map

        # Initialize C_new using the updated formula
        u_shift_x = np.where(u > 0, 1, -1)
        v_shift_y = np.where(v > 0, 1, -1)

        # Calculate the mask for valid neighbors
        valid_u_mask = np.where(np.roll(land_mask, -1, axis=0) == False, 1, 0)
        valid_v_mask = np.where(np.roll(land_mask, -1, axis=1) == False, 1, 0)

        # Calculate C_new with the updated rule
        C_new = (
            (1 - 2 * dt * Dx / dx**2 - 2 * dt * Dy / dy**2)
            - (np.abs(u) * valid_u_mask * dt / dx)
            - (np.abs(v) * valid_v_mask * dt / dy)
        ) * C_current

        # Apply the pollution source term during the first half of iterations
        pollution_source = np.where(iteration < iterations / 2, S * dt, 0)
        C_new += pollution_source

        # Calculate contributions from neighboring cells (same logic as before)
        mask_x_positive = (np.roll(land_mask, -1, axis=0) == False) & (np.arange(rows) + 1 < rows)
        u_x_positive = np.roll(u, -1, axis=0)  # u[i+1] for x+ direction
        C_new = C_new + (dt / dx) * (Dx / dx - np.where(u_x_positive < 0, u_x_positive, 0)) * np.roll(ocean_map, -1, axis=0) * mask_x_positive

        mask_x_negative = (np.roll(land_mask, 1, axis=0) == False) & (np.arange(rows) - 1 >= 0)
        u_x_negative = np.roll(u, 1, axis=0)  # u[i-1] for x- direction
        C_new = C_new + (dt / dx) * (Dx / dx + np.where(u_x_negative > 0, u_x_negative, 0)) * np.roll(ocean_map, 1, axis=0) * mask_x_negative

        mask_y_positive = (np.roll(land_mask, -1, axis=1) == False) & (np.arange(cols) + 1 < cols)
        v_y_positive = np.roll(v, -1, axis=1)  # v[j+1] for y+ direction
        C_new = C_new + (dt / dy) * (Dy / dy - np.where(v_y_positive < 0, v_y_positive, 0)) * np.roll(ocean_map, -1, axis=1) * mask_y_positive

        mask_y_negative = (np.roll(land_mask, 1, axis=1) == False) & (np.arange(cols) - 1 >= 0)
        v_y_negative = np.roll(v, 1, axis=1)  # v[j-1] for y- direction
        C_new = C_new + (dt / dy) * (Dy / dy + np.where(v_y_negative > 0, v_y_negative, 0)) * np.roll(ocean_map, 1, axis=1) * mask_y_negative

        # Update the ocean_map
        ocean_map = np.where(land_mask, 10, C_new)

        if iteration % 10 == 0:
            maps_over_time.append(np.copy(ocean_map))
    return maps_over_time

def update_concentration(ocean_map, land_mask, S, Dx, Dy, u, v, dt, dx, dy, iterations):
    # Get the shape of the map
    rows, cols = ocean_map.shape

    # Initialize a copy of the ocean_map to store updates
    updated_map = np.copy(ocean_map)
    maps_over_time = [np.copy(ocean_map)]

    for _ in range(iterations):
        # Iterate through each cell in the ocean_map
        for i in range(rows):
            for j in range(cols):
                # Skip updating if it's a land cell
                if land_mask[i, j]:
                    continue

                # Get the current concentration value
                C_current = ocean_map[i, j]

                # Use the local Dx, Dy, u, v values
                Dx_ij = Dx[i, j]
                Dy_ij = Dy[i, j]
                u_ij = u[i, j]
                v_ij = v[i, j]

                # Calculate the contribution from the center cell
                sx = 1 if u_ij > 0 else -1
                sy = 1 if v_ij > 0 else -1
                C_new = (1 - 2 * dt * Dx_ij / dx**2 - 2 * dt * Dy_ij / dy**2 - (u_ij * dt / dx) * sx - (v_ij * dt / dy) * sy) * C_current + (S[i, j] * dt if _ < iterations / 4 else 0)
                # Calculate contributions from neighboring cells, ensuring boundary conditions
                # if (S[i, j] > 0) :
                #     print(S[i, j])
                # x + dx
                if i + 1 < rows and not land_mask[i + 1, j]:
                    C_new += (dt / dx) * (Dx_ij / dx - (u_ij if u_ij < 0 else 0)) * ocean_map[i + 1, j]
                    # C_new += (dt / dx) * (Dx_ij / dx - (u_ij / 2)) * ocean_map[i + 1, j]
                else :
                    C_new += (dt / dx) * (Dx_ij / dx - (u_ij if u_ij < 0 else 0)) * C_current

                # x - dx
                if i - 1 >= 0 and not land_mask[i - 1, j]:
                    C_new += (dt / dx) * (Dx_ij / dx + (u_ij if u_ij > 0 else 0)) * ocean_map[i - 1, j]
                    # C_new += (dt / dx) * (Dx_ij / dx + (u_ij / 2)) * ocean_map[i - 1, j]
                else :
                    C_new += (dt / dx) * (Dx_ij / dx + (u_ij if u_ij > 0 else 0)) * C_current

                # y + dy
                if j + 1 < cols and not land_mask[i, j + 1]:
                    C_new += (dt / dy) * (Dy_ij / dy - (v_ij if v_ij < 0 else 0)) * ocean_map[i, j + 1]
                    # C_new += (dt / dy) * (Dy_ij / dy - (v_ij / 2)) * ocean_map[i, j + 1]
                else :
                    C_new += (dt / dy) * (Dy_ij / dy - (v_ij if v_ij < 0 else 0)) * C_current

                # y - dy
                if j - 1 >= 0 and not land_mask[i, j - 1]:
                    C_new += (dt / dy) * (Dy_ij / dy + (v_ij if v_ij > 0 else 0)) * ocean_map[i, j - 1]
                    # C_new += (dt / dy) * (Dy_ij / dy + (v_ij / 2)) * ocean_map[i, j - 1]
                else :
                    C_new += (dt / dy) * (Dy_ij / dy + (v_ij if v_ij > 0 else 0)) * C_current

                # Update the value in the updated_map
                updated_map[i, j] = C_new

        # Update the ocean_map for the next iteration
        ocean_map = np.copy(updated_map)
        if _ % 100 == 0:
            maps_over_time.append(np.copy(ocean_map))

    return maps_over_time

def animate_concentration(maps_over_time):
    fig, ax = plt.subplots(figsize=(6, 6))
    cmap = 'viridis'
    cax = ax.imshow(maps_over_time[0], cmap=cmap, vmax=10)
    fig.colorbar(cax, ax=ax, label='Concentration')

    def update(frame):
        cax.set_data(maps_over_time[frame])
        ax.set_title(f'Concentration Map - Iteration {frame + 1}')

    return animation.FuncAnimation(fig, update, frames=len(maps_over_time), interval=1)


In [ ]:
!wget -O land_image.jpg https://classroom.sanibelseaschool.org/wp-content/uploads/gom.jpg


In [ ]:
# Parameters
rows, cols = 100, 100  # Map size (larger to better visualize)
dt = 0.01  # Time step
dx, dy = 1.0, 1.0  # Spatial step sizes
iterations = 10000  # Number of iterations

# Generate random Dx, Dy, u, v arrays
Dx, Dy, u, v = generate_random_Dx_Dy(rows, cols)

# Generate random inputs with waste processing points
ocean_map, land_mask, S = generate_random_inputs(rows, cols)

# Plot the land_image input (if existed) and land_mask

plt.figure(figsize=(6, 6))
plt.imshow(land_mask, cmap='gray')
plt.title('Land Mask')
plt.show()

In [ ]:
# Update concentration and collect maps over time
#maps_over_time = update_concentration(ocean_map, land_mask, S, Dx, Dy, u, v, dt, dx, dy, iterations)
maps_over_time = update_concentration_parallel(ocean_map, land_mask, S, Dx, Dy, u, v, dt, dx, dy, iterations)
# Animate the concentration map
animation = animate_concentration(maps_over_time)

In [ ]:
animation